# #12 Minimum Volatility Portfolio_scipy.optimize & cvxopt

[reference](http://henryquant.blogspot.com/2017/08/minimum-volatility-portfolio-python.html)

## 1. method using `scipy.optimize`

In [1]:
import pandas as pd
from pandas import Series, DataFrame
from pandas.tseries.offsets import Day, MonthEnd

import numpy as np
import sys

import matplotlib.pyplot as plt

from scipy.stats import rankdata
from scipy.stats import stats
from scipy.optimize import minimize

In [2]:
# setting the dataset
import yfinance as yf
tickers = ['MSFT', 'TLT', 'SPY', 'VTIP', 'VCLT', 'IAU', 'BCI']
df = yf.download(tickers, start='2017-06-01', end='2023-04-01')['Adj Close']
df.head()

[*********************100%***********************]  7 of 7 completed


,BCI,IAU,MSFT,SPY,TLT,VCLT,VTIP
Date,,,,,,,
2017-06-01,15.392171,24.420000,65.030838,218.480835,108.937943,73.038712,41.170410
2017-06-02,15.392171,24.580000,66.570801,219.208023,110.227776,73.720505,41.187046
2017-06-05,15.290108,24.620001,67.053177,219.046448,109.543350,73.363754,41.153759
2017-06-06,15.290108,24.900000,67.275833,218.346176,110.140068,73.736343,41.220341
2017-06-07,15.290108,24.780001,67.155243,218.750183,109.613564,73.585732,41.137115


In [3]:
ret = df.pct_change().dropna() * 100
ret

,BCI,IAU,MSFT,SPY,TLT,VCLT,VTIP
Date,,,,,,,
2017-06-02,0.000000,0.655200,2.368050,0.332838,1.184007,0.933468,0.040407
2017-06-05,-0.663085,0.162738,0.724606,-0.073709,-0.620919,-0.483923,-0.080819
2017-06-06,0.000000,1.137282,0.332059,-0.319691,0.544732,0.507865,0.161788
2017-06-07,0.000000,-0.481923,-0.179247,0.185030,-0.478032,-0.204257,-0.201903
2017-06-08,0.041727,-0.726393,-0.607850,0.049233,-0.264156,-0.172379,-0.060693
...,...,...,...,...,...,...,...
2023-03-27,0.988635,-0.907635,-1.493395,0.186982,-2.349088,-1.673151,-0.272422
2023-03-28,0.783161,0.835133,-0.416083,-0.224465,0.182101,-0.179145,0.021006
2023-03-29,-0.437106,-0.507611,1.918388,1.453483,-0.200901,0.692140,0.105048


In [4]:
cov = df.cov()
cov

,BCI,IAU,MSFT,SPY,TLT,VCLT,VTIP
BCI,10.749686,6.505213,161.215305,165.315246,-23.520091,-4.962049,5.749514
IAU,6.505213,23.186728,341.220232,296.775975,48.636281,34.722918,11.067566
MSFT,161.215305,341.220232,6354.801617,5768.973144,557.264644,503.773453,201.301246
SPY,165.315246,296.775975,5768.973144,5459.413396,374.998495,430.534642,184.230402
TLT,-23.520091,48.636281,557.264644,374.998495,319.920529,168.482332,14.725916
VCLT,-4.962049,34.722918,503.773453,430.534642,168.482332,106.872837,14.602769
VTIP,5.749514,11.067566,201.301246,184.230402,14.725916,14.602769,6.572557


In [5]:
def MinVol(covmat, lb, ub):
    def MinVol_obj(x):
        variance = x.T @ covmat @ x
        sigma = variance ** 0.5  # std dev.
        return sigma

    def weight_sum_constraint(x):
        return (x.sum() - 1.0)

    sz = covmat.shape[1]
    x0 = np.repeat(1/sz, sz)  # (initial) uniform weight
    lbound = np.repeat(lb, sz)
    ubound = np.repeat(ub, sz)
    bnds = tuple(zip(lbound, ubound))

    constraints = ({'type': 'eq', 'fun': weight_sum_constraint})
    options = ({'ftol': 1e-20, 'maxiter': 800})

    result = minimize(fun=MinVol_obj,
                      x0=x0,
                      method='SLSQP',
                      constraints=constraints,
                      options=options,
                      bounds=bnds)
    return result.x

In [6]:
MinVol(cov, 0, 1)

array([1.41338218e-01, 0.00000000e+00, 2.43861058e-13, 4.46077473e-13,
       9.07956243e-13, 0.00000000e+00, 8.58661782e-01])

## 2. method using `cvxopt`

In [ ]:
from cvxopt import matrix
from cvxopt import solvers

def MinVol_cvxopt(covmat, lb, ub):
    sz = len(covmat)

    p = matrix(np.array(covmat), tc='d')
    q = matrix(np.zeros(sz), tc='d')

    lb_diag = np.diag(np.repeat(-1, sz))
    ub_diag = np.diag(np.repeat(+1, sz))

    G = matrix(np.concatenate((lb_diag, ub_diag)), tc='d')
    h = matrix(np.concatenate((np.repeat(-lb, sz)), np.repeat(ub, sz)), tc='d')

    A = matrix(np.repeat(1, sz), tc='d').T
    b = matrix(1, tc='d')

    solution = solvers.qp(P, q, G, h, A, b)
    return np.array(solution['x'])

In [ ]:
MinVol_cvxopt(cov, 0, 1)